# Lista 2 - AVC

O objetivo da lista é praticar queries e operações com DataFrame. Abaixo está o dicionário de dados da tabela.

![a](https://miro.medium.com/max/649/1*3exwRyxaFZuhGB4hk-2K0Q.png)


### Cria uma tabela com DuckDB com os dados do arquivo `healthcare-dataset-stroke-data.csv` que estão no Blackboard

In [1]:
file_path = 'healthcare-dataset-stroke-data.csv'

In [6]:
!pip install duckdb


In [ ]:

import duckdb
import pandas as pd


In [27]:
file_path = 'healthcare-dataset-stroke-data.csv'
data = pd.read_csv(file_path)
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [11]:
conn = duckdb.connect()

conn.execute('''
CREATE TABLE healthcare_data (
    id INTEGER,
    gender STRING,
    age DOUBLE,
    hypertension INTEGER,
    heart_disease INTEGER,
    ever_married STRING,
    work_type STRING,
    Residence_type STRING,
    avg_glucose_level DOUBLE,
    bmi DOUBLE,
    smoking_status STRING,
    stroke INTEGER
)
''')

conn.execute("INSERT INTO healthcare_data SELECT * FROM data")

result = conn.execute("SELECT * FROM healthcare_data LIMIT 5").fetchdf()
print(result)

      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  


### Crie uma query SQL com o número de pessoas com AVC por tipo de trabalho, renomeando a coluna com os valores para `work_type_count`, em ordem decrescente

In [14]:
AVC_query ="""
            SELECT
                work_type,
                COUNT(*) AS work_type_count
            FROM
                healthcare_data
            WHERE
                stroke = 1
            GROUP BY
                work_type
            ORDER BY
                work_type_count DESC;
            """


result = conn.execute(AVC_query)
result = result.fetchdf()
print(result)


       work_type  work_type_count
0        Private              149
1  Self-employed               65
2       Govt_job               33
3       children                2


### Crie uma query SQL com o número total e a porcentagem de cada gênero na base

In [18]:
result = conn.execute("""
            SELECT
                gender,
                COUNT(*) AS total_count,
                ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM healthcare_data)), 2) AS percentage
            FROM
                healthcare_data
            GROUP BY
                gender;
            """
)
result = result.fetchdf()
print(result)

   gender  total_count  percentage
0    Male         2115       41.39
1  Female         2994       58.59
2   Other            1        0.02


### Crie uma query SQL com o número total e a porcentagem de pacientes masculinos com AVC

In [20]:
result = conn.execute("""
                      SELECT
                          COUNT(*) AS total_male_stroke_count,
                          ROUND((COUNT(*) * 100.0 / (SELECT COUNT(*) FROM healthcare_data WHERE gender = 'Male')), 2) AS male_stroke_percentage
                      FROM
                          healthcare_data
                      WHERE
                          gender = 'Male' AND stroke = 1;

            """
)
result = result.fetchdf()
print(result)

   total_male_stroke_count  male_stroke_percentage
0                      108                    5.11


### Crie uma query SQL com a quantidade de AVCs por faixa etária

In [24]:
resutl = conn.execute(
                      """
                      SELECT
                          CASE
                              WHEN age BETWEEN 0 AND 18 THEN '0-18'
                              WHEN age BETWEEN 19 AND 35 THEN '19-35'
                              WHEN age BETWEEN 36 AND 50 THEN '36-50'
                              WHEN age BETWEEN 51 AND 65 THEN '51-65'
                              WHEN age BETWEEN 66 AND 80 THEN '66-80'
                              WHEN age > 80 THEN '81+'
                          END AS age_range,
                          COUNT(*) AS stroke_count
                      FROM
                          healthcare_data
                      WHERE
                          stroke = 1
                      GROUP BY
                          age_range
                      ORDER BY
                          age_range;
                      """
)

print(result)
conn.close()

   total_male_stroke_count  male_stroke_percentage
0                      108                    5.11
